In [1]:
import torch
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from torch.utils.tensorboard import SummaryWriter
import pytorch_lightning as pl
import random
import dotenv
import omegaconf
import hydra
import logging
from typing import List

import wandb
from datetime import date
import dotenv
import os
import pathlib
from typing import Dict, Any
from copy import deepcopy

from rigl_torch.models import ModelFactory
from rigl_torch.rigl_scheduler import RigLScheduler
from rigl_torch.rigl_constant_fan import RigLConstFanScheduler
from rigl_torch.datasets import get_dataloaders
from rigl_torch.optim import (
    get_optimizer,
    get_lr_scheduler,
)
from rigl_torch.utils.rigl_utils import get_names_and_W
from rigl_torch.utils.checkpoint import Checkpoint
from rigl_torch.utils.rigl_utils import get_T_end, get_fan_in_after_ablation, get_conv_idx_from_flat_idx
from hydra import initialize, compose

from fvcore.nn import FlopCountAnalysis
import pandas as pd


In [2]:
def get_pruner_model_loader(dense_alloc, model, dataset):
    with initialize("../configs", version_base="1.2.0"):
        cfg = compose(
            "config.yaml",
            overrides=[
                f"dataset={dataset}",
                "compute.distributed=False",
                f"model={model}",
                f"rigl.dense_allocation={dense_alloc}",
                ])
    dotenv.load_dotenv("../.env")
    os.environ["IMAGE_NET_PATH"]


    rank=0
    checkpoint=None
    if checkpoint is not None:
        run_id = checkpoint.run_id
        optimizer_state = checkpoint.optimizer
        scheduler_state = checkpoint.scheduler
        pruner_state = checkpoint.pruner
        model_state = checkpoint.model
        cfg = checkpoint.cfg
    else:
        run_id, optimizer_state, scheduler_state, pruner_state, model_state = (
            None,
            None,
            None,
            None,
            None,
        )

    print(cfg.compute)
    cfg.compute.distributed=False
        
    pl.seed_everything(cfg.training.seed)
    use_cuda = not cfg.compute.no_cuda and torch.cuda.is_available()
    if not use_cuda:
        raise SystemError("GPU has stopped responding...waiting to die!")
        logger.warning(
            "Using CPU! Verify cfg.compute.no_cuda and "
            "torch.cuda.is_available() are properly set if this is unexpected"
        )

    if cfg.compute.distributed and use_cuda:
        device = torch.device(f"cuda:{rank}")
    else:
        print(f"loading to device rank: {rank}")
        device = torch.device(f"cuda:{rank}")
    if not use_cuda:
        device = torch.device("cuda" if use_cuda else "cpu")
    train_loader, test_loader = get_dataloaders(cfg)

    model = ModelFactory.load_model(
        model=cfg.model.name, dataset=cfg.dataset.name
    )
    model.to(device)
    if cfg.compute.distributed:
        model = DistributedDataParallel(model, device_ids=[rank])
    if model_state is not None:
        try:
            model.load_state_dict(model_state)
        except RuntimeError:
            model_state = checkpoint.get_single_process_model_state_from_distributed_state()
            model.load_state_dict(model_state)
            
    optimizer = get_optimizer(cfg, model, state_dict=optimizer_state)
    scheduler = get_lr_scheduler(cfg, optimizer, state_dict=scheduler_state)
    pruner = None
    if cfg.rigl.dense_allocation is not None:
        T_end = get_T_end(cfg, [0 for _ in range(0,1251)])
        if cfg.rigl.const_fan_in:
            rigl_scheduler = RigLConstFanScheduler
        else:
            rigl_scheduler = RigLScheduler
        pruner = rigl_scheduler(
            model,
            optimizer,
            dense_allocation=cfg.rigl.dense_allocation,
            alpha=cfg.rigl.alpha,
            delta=cfg.rigl.delta,
            static_topo=cfg.rigl.static_topo,
            T_end=T_end,
            ignore_linear_layers=cfg.rigl.ignore_linear_layers,
            grad_accumulation_n=cfg.rigl.grad_accumulation_n,
            sparsity_distribution=cfg.rigl.sparsity_distribution,
            erk_power_scale=cfg.rigl.erk_power_scale,
            state_dict=pruner_state,
            filter_ablation_threshold=cfg.rigl.filter_ablation_threshold,
            static_ablation=cfg.rigl.static_ablation,
            dynamic_ablation=cfg.rigl.dynamic_ablation,
            min_salient_weights_per_neuron=cfg.rigl.min_salient_weights_per_neuron,  # noqa
            use_sparse_init=cfg.rigl.use_sparse_initialization,
            init_method_str=cfg.rigl.init_method_str,
            use_sparse_const_fan_in_for_ablation=cfg.rigl.use_sparse_const_fan_in_for_ablation,  # noqa
        )
        
        step=0
        return pruner, model, train_loader

In [3]:
def get_flops_df(model_name, dataset):
    df = {k:[] for k in ["rigl.dense_allocation", "flops", "model",]}
    for da in [0.01, 0.05, 0.0625, 0.1, 0.2, 0.25,]:
        print(f"Calculating with dense_alloc == {da}")
        pruner, model, train_loader = get_pruner_model_loader(da, model_name, dataset)
        model.eval()
        for data, _ in train_loader:
            data = data[0].to("cpu").reshape(1, *data[0].shape)
            break
        flops = FlopCountAnalysis(model.to("cpu"),data)
        total_flops = 0
        S = pruner.S
        names, W = get_names_and_W(model)
        for name, counter in flops.by_module_and_operator().items():
            if name in names:
                if len(counter) != 1:
                    raise ValueError(f"Too many items found in {name}. Goodbye")
                f = list(counter.values())[0]
                s = S[names.index(name)]
                if s is None:
                    s=1
                total_flops += f*(1-s)
        del model
        del pruner
        del train_loader
        df["rigl.dense_allocation"].append(da)
        df["flops"].append(total_flops)
        df["model"].append(model_name)
    return pd.DataFrame(df)

# df = get_flops_df("resnet50", "imagenet")

In [13]:
p, m, l = get_pruner_model_loader(0.01, "resnet50", "imagenet")

Global seed set to 42


{'no_cuda': False, 'cuda_kwargs': {'num_workers': '${ oc.decode:${oc.env:NUM_WORKERS} }', 'pin_memory': True}, 'distributed': False, 'world_size': 4, 'dist_backend': 'nccl'}
loading to device rank: 0


INFO:/home/mike/condensed-sparsity/src/rigl_torch/models/model_factory.py:Loading model resnet50/imagenet using <function get_imagenet_resnet50 at 0x7f9881d5a200> with args: () and kwargs: {}


In [10]:
from micronet_challenge.counting import *
import torch.nn as nn
Conv2D._fields
 
def get_conv_op(conv: nn.Conv2d, input):
    return Conv2D(
        input_size=input.shape[1],
        kernel_shape=
    )
    

# def get_op()


# for name, child in model.named_children():
#     print(type(child))
#     break
# # names, _ = get_names_and_W(model)
# type(w[0])

('input_size', 'kernel_shape', 'strides', 'padding', 'use_bias', 'activation')

In [46]:
from typing import List, Optional

def get_names_and_children(
    module,
    parents: Optional[List] = None,
    name_append:Optional[str] = None
):
    if parents is None:
        parents =[]
    
    for name, child in module.named_children():
    
        return child
        # if len(child._mod)
        print(child._modules)

child = get_names_and_children(m)
child.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_load_state_dict_post_hooks', '_modules', 'in_channels', 'out_channels', 'kernel_size', 'stride', 'padding', 'dilation', 'transposed', 'output_padding', 'groups', 'padding_mode', '_reversed_padding_repeated_twice'])

In [47]:
type(child)

torch.nn.modules.conv.Conv2d

In [42]:
child.__dict__["_modules"]

OrderedDict()

In [26]:
target_types = [torch.nn.Conv2d, torch.nn.Linear]

In [25]:
modules.Linear

torch.nn.modules.linear.Linear

In [22]:
modules.conv.Conv2d

torch.nn.modules.conv.Conv2d

In [21]:
df.by_operator()["conv"]/df.total()

0.9940712038332824

In [19]:
df.by_module_and_operator()

{'': Counter({'conv': 4087136256,
          'batch_norm': 22227968,
          'adaptive_avg_pool2d': 100352,
          'linear': 2048000}),
 'conv1': Counter({'conv': 118013952}),
 'bn1': Counter({'batch_norm': 1605632}),
 'relu': Counter(),
 'maxpool': Counter(),
 'layer1': Counter({'conv': 667942912, 'batch_norm': 8830976}),
 'layer1.0': Counter({'conv': 231211008, 'batch_norm': 4014080}),
 'layer1.0.conv1': Counter({'conv': 12845056}),
 'layer1.0.bn1': Counter({'batch_norm': 401408}),
 'layer1.0.conv2': Counter({'conv': 115605504}),
 'layer1.0.bn2': Counter({'batch_norm': 401408}),
 'layer1.0.conv3': Counter({'conv': 51380224}),
 'layer1.0.bn3': Counter({'batch_norm': 1605632}),
 'layer1.0.relu': Counter(),
 'layer1.0.downsample': Counter({'conv': 51380224, 'batch_norm': 1605632}),
 'layer1.0.downsample.0': Counter({'conv': 51380224}),
 'layer1.0.downsample.1': Counter({'batch_norm': 1605632}),
 'layer1.1': Counter({'conv': 218365952, 'batch_norm': 2408448}),
 'layer1.1.conv1': Coun

In [88]:
w_idx = 0
total_flops = 0
modules_to_ignore = []
S = pruner.S
names, W = get_names_and_W(model)
for name, counter in flops.by_module_and_operator().items():
    if name in names:
        if len(counter) != 1:
            raise ValueError("?")
        f = list(counter.values())[0]
        s = S[names.index(name)]
        if s is None:
            s=1
        total_flops += f*(1-s)

total_flops

99517038.63962848

In [91]:
(99517038.63962848 / 4089284608)*100

2.4336050967188756

In [84]:
3989667217/1e9

3.989667217

In [85]:
4089284608/1e9

4.089284608

In [83]:
4089284608-3989667217.360372

99617390.63962793

In [51]:
total - flops.by_operator()['batch_norm']

4089284608

In [45]:
total - flops.by_operator()['batch_norm']

4089284608

In [46]:
4089284608/1e9

4.089284608

4089284608/1e9

In [41]:
4111512576/1e9

4.111512576

In [40]:
4111512576 - 3.8e9

311512576.0

In [22]:
from fvcore.nn import flop_count_table

print(flop_count_table(flops))

| module                 | #parameters or shape   | #flops     |
|:-----------------------|:-----------------------|:-----------|
| model                  | 25.557M                | 0.531T     |
|  conv1                 |  9.408K                |  15.106G   |
|   conv1.weight         |   (64, 3, 7, 7)        |            |
|  bn1                   |  0.128K                |  0.514G    |
|   bn1.weight           |   (64,)                |            |
|   bn1.bias             |   (64,)                |            |
|  layer1                |  0.216M                |  88.323G   |
|   layer1.0             |   75.008K              |   30.88G   |
|    layer1.0.conv1      |    4.096K              |    1.644G  |
|    layer1.0.bn1        |    0.128K              |    0.128G  |
|    layer1.0.conv2      |    36.864K             |    14.798G |
|    layer1.0.bn2        |    0.128K              |    0.128G  |
|    layer1.0.conv3      |    16.384K             |    6.577G  |
|    layer1.0.bn3        

In [8]:
from ptflops import get_model_complexity_info
macs, params = get_model_complexity_info(model, (3, 224, 224), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print(macs)

ResNet(
  25.56 M, 100.000% Params, 4.12 GMac, 100.000% MACs, 
  (conv1): Conv2d(9.41 k, 0.037% Params, 118.01 MMac, 2.863% MACs, 3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(128, 0.001% Params, 1.61 MMac, 0.039% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(0, 0.000% Params, 802.82 KMac, 0.019% MACs, inplace=True)
  (maxpool): MaxPool2d(0, 0.000% Params, 802.82 KMac, 0.019% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    215.81 k, 0.844% Params, 680.39 MMac, 16.507% MACs, 
    (0): Bottleneck(
      75.01 k, 0.293% Params, 236.43 MMac, 5.736% MACs, 
      (conv1): Conv2d(4.1 k, 0.016% Params, 12.85 MMac, 0.312% MACs, 64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, 0.001% Params, 401.41 KMac, 0.010% MACs, 64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(36.86 k, 0.144% Pa

In [ ]:
 checkpoint=None
    rank=1

    if checkpoint is not None:
        run_id = checkpoint.run_id
        optimizer_state = checkpoint.optimizer
        scheduler_state = checkpoint.scheduler
        pruner_state = checkpoint.pruner
        model_state = checkpoint.model
        cfg = checkpoint.cfg
    else:
        run_id, optimizer_state, scheduler_state, pruner_state, model_state = (
            None,
            None,
            None,
            None,
            None,
        )

    print(cfg.compute)
    cfg.compute.distributed=False
        
    pl.seed_everything(cfg.training.seed)
    use_cuda = not cfg.compute.no_cuda and torch.cuda.is_available()
    if not use_cuda:
        raise SystemError("GPU has stopped responding...waiting to die!")
        logger.warning(
            "Using CPU! Verify cfg.compute.no_cuda and "
            "torch.cuda.is_available() are properly set if this is unexpected"
        )

    if cfg.compute.distributed and use_cuda:
        device = torch.device(f"cuda:{rank}")
    else:
        print(f"loading to device rank: {rank}")
        device = torch.device(f"cuda:{rank}")
    if not use_cuda:
        device = torch.device("cuda" if use_cuda else "cpu")
    # train_loader, test_loader = get_dataloaders(cfg)

    model = ModelFactory.load_model(
        model=cfg.model.name, dataset=cfg.dataset.name, diet=cfg.rigl.diet
    )
    model.to(device)
    if cfg.compute.distributed:
        model = DistributedDataParallel(model, device_ids=[rank])
    if model_state is not None:
        try:
            model.load_state_dict(model_state)
        except RuntimeError:
            model_state = checkpoint.get_single_process_model_state_from_distributed_state()
            model.load_state_dict(model_state)
            
    optimizer = get_optimizer(cfg, model, state_dict=optimizer_state)
    scheduler = get_lr_scheduler(cfg, optimizer, state_dict=scheduler_state)
    pruner = None
    if cfg.rigl.dense_allocation is not None:
        if cfg.rigl.dense_allocation is not None:
            if cfg.model.name == "skinny_resnet18":
                dense_allocation = (
                    cfg.rigl.dense_allocation * cfg.model.sparsity_scale_factor
                )
                print(
                    f"Scaling {cfg.rigl.dense_allocation} by "
                    f"{cfg.model.sparsity_scale_factor:.2f} for SkinnyResNet18 "
                    f"New Dense Alloc == {dense_allocation:.6f}"
                )
            else:
                dense_allocation = cfg.rigl.dense_allocation
            T_end = get_T_end(cfg, [0 for _ in range(0,1251)])
            if cfg.rigl.const_fan_in:
                rigl_scheduler = RigLConstFanScheduler
            else:
                rigl_scheduler = RigLScheduler
            pruner = rigl_scheduler(
                model,
                optimizer,
                dense_allocation=da,
                alpha=cfg.rigl.alpha,
                delta=cfg.rigl.delta,
                static_topo=cfg.rigl.static_topo,
                T_end=T_end,
                ignore_linear_layers=cfg.rigl.ignore_linear_layers,
                grad_accumulation_n=cfg.rigl.grad_accumulation_n,
                sparsity_distribution=cfg.rigl.sparsity_distribution,
                erk_power_scale=cfg.rigl.erk_power_scale,
                state_dict=pruner_state,
                filter_ablation_threshold=cfg.rigl.filter_ablation_threshold,
                static_ablation=cfg.rigl.static_ablation,
                dynamic_ablation=cfg.rigl.dynamic_ablation,
                min_salient_weights_per_neuron=cfg.rigl.min_salient_weights_per_neuron,  # noqa
                use_sparse_init=cfg.rigl.use_sparse_initialization,
                init_method_str=cfg.rigl.init_method_str,
                use_sparse_const_fan_in_for_ablation=cfg.rigl.use_sparse_const_fan_in_for_ablation,  # noqa
            )
            step=0
    print(f"Results for {1-da} sparsity...")
    els = []
    non_zero=[]
    for p in model.parameters():
        els.append(p.numel())
        non_zero.append(torch.count_nonzero(p))
    s=torch.tensor(non_zero).sum().item()
    n=torch.tensor(els).sum().item()
    w_total = torch.tensor([torch.count_nonzero(w) for w in pruner.W]).sum()
    params["dense_allocation"].append(da)
    params["parameters"].append(s)
    params["dense_params"].append(n)
    # print(f"total el: {n}")
    # print(f"non zero el: {s}")
    # print(f"non zero weights only: {w_total}")
    # print(s/n)

In [34]:
params = {k:[] for k in ["dense_allocation", "parameters", "dense_params"]}
for da in [0.01, 0.05, 0.0625, 0.1, 0.2, 0.25, 0.3, 0.4, 0.5,]:
    checkpoint=None
    rank=1

    if checkpoint is not None:
        run_id = checkpoint.run_id
        optimizer_state = checkpoint.optimizer
        scheduler_state = checkpoint.scheduler
        pruner_state = checkpoint.pruner
        model_state = checkpoint.model
        cfg = checkpoint.cfg
    else:
        run_id, optimizer_state, scheduler_state, pruner_state, model_state = (
            None,
            None,
            None,
            None,
            None,
        )

    print(cfg.compute)
    cfg.compute.distributed=False
        
    pl.seed_everything(cfg.training.seed)
    use_cuda = not cfg.compute.no_cuda and torch.cuda.is_available()
    if not use_cuda:
        raise SystemError("GPU has stopped responding...waiting to die!")
        logger.warning(
            "Using CPU! Verify cfg.compute.no_cuda and "
            "torch.cuda.is_available() are properly set if this is unexpected"
        )

    if cfg.compute.distributed and use_cuda:
        device = torch.device(f"cuda:{rank}")
    else:
        print(f"loading to device rank: {rank}")
        device = torch.device(f"cuda:{rank}")
    if not use_cuda:
        device = torch.device("cuda" if use_cuda else "cpu")
    # train_loader, test_loader = get_dataloaders(cfg)

    model = ModelFactory.load_model(
        model=cfg.model.name, dataset=cfg.dataset.name, diet=cfg.rigl.diet
    )
    model.to(device)
    if cfg.compute.distributed:
        model = DistributedDataParallel(model, device_ids=[rank])
    if model_state is not None:
        try:
            model.load_state_dict(model_state)
        except RuntimeError:
            model_state = checkpoint.get_single_process_model_state_from_distributed_state()
            model.load_state_dict(model_state)
            
    optimizer = get_optimizer(cfg, model, state_dict=optimizer_state)
    scheduler = get_lr_scheduler(cfg, optimizer, state_dict=scheduler_state)
    pruner = None
    if cfg.rigl.dense_allocation is not None:
        if cfg.rigl.dense_allocation is not None:
            if cfg.model.name == "skinny_resnet18":
                dense_allocation = (
                    cfg.rigl.dense_allocation * cfg.model.sparsity_scale_factor
                )
                print(
                    f"Scaling {cfg.rigl.dense_allocation} by "
                    f"{cfg.model.sparsity_scale_factor:.2f} for SkinnyResNet18 "
                    f"New Dense Alloc == {dense_allocation:.6f}"
                )
            else:
                dense_allocation = cfg.rigl.dense_allocation
            T_end = get_T_end(cfg, [0 for _ in range(0,1251)])
            if cfg.rigl.const_fan_in:
                rigl_scheduler = RigLConstFanScheduler
            else:
                rigl_scheduler = RigLScheduler
            pruner = rigl_scheduler(
                model,
                optimizer,
                dense_allocation=da,
                alpha=cfg.rigl.alpha,
                delta=cfg.rigl.delta,
                static_topo=cfg.rigl.static_topo,
                T_end=T_end,
                ignore_linear_layers=cfg.rigl.ignore_linear_layers,
                grad_accumulation_n=cfg.rigl.grad_accumulation_n,
                sparsity_distribution=cfg.rigl.sparsity_distribution,
                erk_power_scale=cfg.rigl.erk_power_scale,
                state_dict=pruner_state,
                filter_ablation_threshold=cfg.rigl.filter_ablation_threshold,
                static_ablation=cfg.rigl.static_ablation,
                dynamic_ablation=cfg.rigl.dynamic_ablation,
                min_salient_weights_per_neuron=cfg.rigl.min_salient_weights_per_neuron,  # noqa
                use_sparse_init=cfg.rigl.use_sparse_initialization,
                init_method_str=cfg.rigl.init_method_str,
                use_sparse_const_fan_in_for_ablation=cfg.rigl.use_sparse_const_fan_in_for_ablation,  # noqa
            )
            step=0
    print(f"Results for {1-da} sparsity...")
    els = []
    non_zero=[]
    for p in model.parameters():
        els.append(p.numel())
        non_zero.append(torch.count_nonzero(p))
    s=torch.tensor(non_zero).sum().item()
    n=torch.tensor(els).sum().item()
    w_total = torch.tensor([torch.count_nonzero(w) for w in pruner.W]).sum()
    params["dense_allocation"].append(da)
    params["parameters"].append(s)
    params["dense_params"].append(n)
    # print(f"total el: {n}")
    # print(f"non zero el: {s}")
    # print(f"non zero weights only: {w_total}")
    # print(s/n)

Global seed set to 42
INFO:/home/user/condensed-sparsity/src/rigl_torch/models/model_factory.py:Loading model resnet50/imagenet using <function get_imagenet_resnet50 at 0x7f5a9bebf910> with args: () and kwargs: {'diet': [17, 16, 17, 14, 16, 33, 31, 31, 32, 30, 65, 63, 63, 65, 63, 122, 124, 124, 114, 125, 0]}


{'no_cuda': False, 'cuda_kwargs': {'num_workers': '${ oc.decode:${oc.env:NUM_WORKERS} }', 'pin_memory': True}, 'distributed': False, 'world_size': 4, 'dist_backend': 'nccl'}
loading to device rank: 1


Global seed set to 42
INFO:/home/user/condensed-sparsity/src/rigl_torch/models/model_factory.py:Loading model resnet50/imagenet using <function get_imagenet_resnet50 at 0x7f5a9bebf910> with args: () and kwargs: {'diet': [17, 16, 17, 14, 16, 33, 31, 31, 32, 30, 65, 63, 63, 65, 63, 122, 124, 124, 114, 125, 0]}


Results for 0.99 sparsity...
{'no_cuda': False, 'cuda_kwargs': {'num_workers': '${ oc.decode:${oc.env:NUM_WORKERS} }', 'pin_memory': True}, 'distributed': False, 'world_size': 4, 'dist_backend': 'nccl'}
loading to device rank: 1


Global seed set to 42
INFO:/home/user/condensed-sparsity/src/rigl_torch/models/model_factory.py:Loading model resnet50/imagenet using <function get_imagenet_resnet50 at 0x7f5a9bebf910> with args: () and kwargs: {'diet': [17, 16, 17, 14, 16, 33, 31, 31, 32, 30, 65, 63, 63, 65, 63, 122, 124, 124, 114, 125, 0]}


...

Results for 0.7 sparsity...
{'no_cuda': False, 'cuda_kwargs': {'num_workers': '${ oc.decode:${oc.env:NUM_WORKERS} }', 'pin_memory': True}, 'distributed': False, 'world_size': 4, 'dist_backend': 'nccl'}
loading to device rank: 1


INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 1 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 3 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 4 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 5 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 7 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 8 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 10 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 11 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 13 set to 0.0
INFO:/home/user/condensed

Results for 0.6 sparsity...
{'no_cuda': False, 'cuda_kwargs': {'num_workers': '${ oc.decode:${oc.env:NUM_WORKERS} }', 'pin_memory': True}, 'distributed': False, 'world_size': 4, 'dist_backend': 'nccl'}
loading to device rank: 1


INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 1 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 3 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 4 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 5 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 7 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 8 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 10 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 11 set to 0.0
INFO:/home/user/condensed-sparsity/src/rigl_torch/rigl_scheduler.py:Sparsity of layer at index 13 set to 0.0
INFO:/home/user/condensed

Results for 0.5 sparsity...


In [35]:
import pandas as pd
pd.DataFrame(params).to_dict()

{'dense_allocation': {0: 0.01,
  1: 0.05,
  2: 0.0625,
  3: 0.1,
  4: 0.2,
  5: 0.25,
  6: 0.3,
  7: 0.4,
  8: 0.5},
 'parameters': {0: 274072,
  1: 1289784,
  2: 1611784,
  3: 2571656,
  4: 5113920,
  5: 6391944,
  6: 7663032,
  7: 10222975,
  8: 12774296},
 'dense_params': {0: 25557032,
  1: 25557032,
  2: 25557032,
  3: 25557032,
  4: 25557032,
  5: 25557032,
  6: 25557032,
  7: 25557032,
  8: 25557032}}